<a href="https://colab.research.google.com/github/antonum/LLM-Document-Chat/blob/main/redis-llm-doc-chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Redis and OpenAI to chat with PDF documents

This notebook demonstrates how to use RedisAI and OpenAI to chat with PDF documents. The PDF included is
a informational brochure about the Chevy Colorado pickup truck.

In this notebook, we will use LLamaIndex to chunk, vectorize, and store the PDF document in Redis as vectors
alongside associated text. The query interface provided by LLamaIndex will be used to search for relevant
information given queries from the user.

##Please provide OpenAI API key

In [1]:
import os
import getpass
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if OPENAI_API_KEY == "" :
  key=getpass.getpass(prompt='OpenAI Key: ', stream=None)
  os.environ['OPENAI_API_KEY']= key

Install dependencies, including local Redis Stack Server and download data.

In [2]:
# Install the requirements
!pip install -q redis PyPDF2 

In [3]:
!pip install -q git+https://github.com/redisventures/llama_index@redis_vector

In [4]:
%%sh
if [ ! -d "docs" ]
then
  mkdir docs
  wget https://www.chevrolet.com/content/dam/chevrolet/na/us/english/index/shopping-tools/download-catalog/03-pdf/2022-chevrolet-colorado-ebrochure.pdf -P docs
fi

Install Redis-Stack-Server only if REDIS_HOST is not set

In [5]:
%%sh
if [ -z "$REDIS_HOST" ]
then
  curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
  echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
  sudo apt-get update
  sudo apt-get install redis-stack-server
  redis-stack-server --daemonize yes
fi


In [6]:
# Import
import textwrap
from IPython.display import display, Markdown
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import RedisVectorStore

### LLamaIndex

[LlamaIndex](https://github.com/jerryjliu/llama_index) (GPT Index) is a project that provides a central interface to connect your LLM's with external data sources. It provides a simple interface to vectorize and store embeddings in Redis, create search indices using Redis, and perform vector search to find context for generative models like GPT.

Here we will use it to load in the documents (Chevy Colorado Brochure).

In [7]:
# load documents
documents = SimpleDirectoryReader('./docs').load_data()
print('Document ID:', documents[0].doc_id, 'Document Hash:', documents[0].doc_hash)

Document ID: 614a0950-88a9-43fe-ab4f-a1b40f792f73 Document Hash: 958a61679fec883f58d6d490edebe15d4bd473e121e03057295d5dda81584204


### Initialize Redis as a Vector Database

Now we have our documents read in, we can initialize the ``RedisVectorStore``. This will allow us to store our vectors in Redis and create an index.

The ``GPTVectorStoreIndex`` will then create the embeddings from the text chunks by calling out to OpenAI's API. The embeddings will be stored in Redis and an index will be created.

NOTE: If you didn't set the ``OPENAI_API_KEY`` environment variable, you will get an error here.

In [8]:
from llama_index.storage.storage_context import StorageContext

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")

vector_store = RedisVectorStore(
    index_name="chevy_docs",
    index_prefix="llama",
    redis_url="redis://default:{}@{}:{}".format(REDIS_PASSWORD,REDIS_HOST,REDIS_PORT),
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)

## Start Querying information from the Document

Now that we have our document stored in the index, we can ask questions against the index. The index will use the data stored in itself as the knowledge base for chatgpt.

In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("What types of variants are available for the Chevrolet Colorado?")
print(textwrap.fill(str(response), 100))

 The Chevrolet Colorado is available in four models: WT, LT, Z71, and ZR2. It is available in both
Extended Cab and Crew Cab configurations, and offers three engine choices: 2.5L 4-cylinder, 3.6L V6,
and Duramax 2.8L Turbo-Diesel. It also offers a variety of features, including Apple CarPlay and
Android Auto compatibility, ZR2 Bison Edition, ZR2 Dusk Special Edition, and ZR2 Midnight Special
Edition.


In [10]:
response = query_engine.query("What is the maximum towing capacity of the chevy colorado?")
print(textwrap.fill(str(response), 100))

 The maximum towing capacity of the Chevy Colorado is 7,700 lbs. with the available Duramax 2.8L
Turbo-Diesel engine.


In [11]:
response = query_engine.query("What are the main differences between the three engine types available for the Chevy Colorado? Format response as a table with model as a first column")
#print(textwrap.fill(str(response), 100))
#print(response)
display(Markdown(str(response)))


Model | Engine | Max Payload Rating (lbs.) | Max Trailering Weight Rating (lbs.)
---- | ---- | ---- | ----
WT | 2.5L I-4 | 1,530 | 37,000
WT | 3.6L V6 | 1,500 | 37,000
LT | 2.5L I-4 | 1,530 | 37,000
LT | 3.6L V6 | 1,490 | 37,000
LT | Duramax 2.8L I-4 | 1,540 | 37,700
Z71 | 3.6L V6 | 1,460 | 7,000
ZR2 | 3.6L V6 | 1,350 | 55,000
ZR2 | Duramax 2.8L I-4 | 1,240 | 55,000